In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
!unzip /kaggle/input/quora-question-pairs/train.csv.zip 

In [ ]:
!ls -l

In [ ]:
data = pd.read_csv('train.csv')

In [ ]:
data

In [ ]:
# Ref: https://hub.packtpub.com/use-tensorflow-and-nlp-to-detect-duplicate-quora-questions-tutorial/
# adding new features

data['len_q1'] = data.question1.apply(lambda x: len(str(x)))
data['len_q2'] = data.question2.apply(lambda x: len(str(x)))
data['diff_len'] = data.len_q1 - data.len_q2

In [ ]:
data['len_char_q1'] = data.question1.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
data['len_char_q2'] = data.question2.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
data['len_word_q1'] = data.question1.apply(lambda x: len(str(x).split(' ')))
data['len_word_q2'] = data.question2.apply(lambda x: len(str(x).split(' ')))

# common words
data['common_words'] = data.apply(
    lambda x: 
    len(set(str(x['question1']).lower().split())
        .intersection(
            set(str(x['question2']).lower().split()))), 
    axis=1)

In [ ]:
fs_1 = ['len_q1', 'len_q2', 'diff_len', 'len_char_q1', 
        'len_char_q2', 'len_word_q1', 'len_word_q2',     
        'common_words']

In [ ]:
!pip install fuzzywuzzy python-Levenshtein

In [ ]:
# using fuzzywuzzy package to find levenshtein distance between question pairs
from fuzzywuzzy import fuzz

In [ ]:
# example of calculating fuzzy number
fuzz.QRatio("How can I start an online shopping (e-commerce) website?", "Which web technology is best suitable for building a big E-Commerce website?")


In [ ]:
fuzz.partial_ratio("Why did Trump win the Presidency?", 
                   "How did Donald Trump win the 2016 Presidential Election")


In [ ]:
fuzz.partial_ratio("How can I start an online shopping (e-commerce) website?", 
                   "Which web technology is best suitable for building a big E-Commerce website?")


In [ ]:
def apply_fuzz(data, column_name, function):
    data[column_name] = data.apply(
        lambda x: function(str(x['question1']), str(x['question2'])), axis=1
    )

In [ ]:
apply_fuzz(data, 'fuzz_ratio', fuzz.QRatio)
apply_fuzz(data, 'fuzz_WRatio', fuzz.WRatio)
apply_fuzz(data, 'fuzz_partial_ratio', fuzz.partial_ratio)
apply_fuzz(data, 'fuzz_partial_token_set_ratio', fuzz.partial_token_set_ratio)
apply_fuzz(data, 'fuzz_partial_token_sort_ratio', fuzz.partial_token_sort_ratio)
apply_fuzz(data, 'fuzz_token_set_ratio', fuzz.token_set_ratio)
apply_fuzz(data, 'fuzz_token_sort_ratio', fuzz.token_sort_ratio)

In [ ]:
fs_2 = ['fuzz_qratio', 'fuzz_WRatio', 'fuzz_partial_ratio', 
       'fuzz_partial_token_set_ratio', 'fuzz_partial_token_sort_ratio',
       'fuzz_token_set_ratio', 'fuzz_token_sort_ratio']

In [ ]:
data

In [ ]:
# tf-idf implementation
from sklearn.feature_extraction.text import TfidfVectorizer
from copy import deepcopy

tfv_q1 = TfidfVectorizer(
    min_df=3, 
    max_features=None, 
    strip_accents='unicode', 
    analyzer='word', 
    token_pattern=r'\w{1,}',
    ngram_range=(1, 2), 
    use_idf=1, 
    smooth_idf=1, 
    sublinear_tf=1,
    stop_words='english')

tfv_q2 = TfidfVectorizer(
    min_df=3, 
    max_features=None, 
    strip_accents='unicode', 
    analyzer='word', 
    token_pattern=r'\w{1,}',
    ngram_range=(1, 2), 
    use_idf=1, 
    smooth_idf=1, 
    sublinear_tf=1,
    stop_words='english')


In [ ]:
q1_tfidf = tfv_q1.fit_transform(data.question1.fillna(""))
q2_tfidf = tfv_q2.fit_transform(data.question2.fillna(""))

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd_q1 = TruncatedSVD(n_components=180)
svd_q2 = TruncatedSVD(n_components=180)

In [ ]:
question1_vectors = svd_q1.fit_transform(q1_tfidf)
question2_vectors = svd_q2.fit_transform(q2_tfidf)

In [ ]:
from scipy import sparse
# obtain features by stacking the sparse matrices together
fs3_1 = sparse.hstack((q1_tfidf, q2_tfidf))

In [ ]:
tfv = TfidfVectorizer(min_df=3, 
                      max_features=None, 
                      strip_accents='unicode', 
                      analyzer='word', 
                      token_pattern=r'\w{1,}',
                      ngram_range=(1, 2), 
                      use_idf=1, 
                      smooth_idf=1, 
                      sublinear_tf=1,
                      stop_words='english')

In [ ]:
# combine questions and calculate tf-idf
q1q2 = data.question1.fillna("") 
q1q2 += " " + data.question2.fillna("")
fs3_2 = tfv.fit_transform(q1q2)

In [ ]:
fs3_3 = np.hstack((question1_vectors, question2_vectors))

In [ ]:
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
from nltk import word_tokenize
stop_words = set(stopwords.words('english'))

def sent2vec(s, model): 
    M = []
    words = word_tokenize(str(s).lower())
    for word in words:
    #It shouldn't be a stopword
        if word not in stop_words:
            #nor contain numbers
            if word.isalpha():
                #and be part of word2vec
                if word in model:
                    M.append(model[word])
    M = np.array(M)
    if len(M) > 0:
        v = M.sum(axis=0)
        return v / np.sqrt((v ** 2).sum())
    else:
        return np.zeros(300)

In [ ]:
model = wv

In [ ]:
w2v_q1 = np.array([sent2vec(q, model) 
                   for q in data.question1])
w2v_q2 = np.array([sent2vec(q, model) 
                   for q in data.question2])

In [ ]:
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis

In [ ]:
data['cosine_distance'] = [cosine(x,y) 
for (x,y) in zip(w2v_q1, w2v_q2)]
data['cityblock_distance'] = [cityblock(x,y) 
for (x,y) in zip(w2v_q1, w2v_q2)]
data['jaccard_distance'] = [jaccard(x,y) 
for (x,y) in zip(w2v_q1, w2v_q2)]
data['canberra_distance'] = [canberra(x,y) 
for (x,y) in zip(w2v_q1, w2v_q2)]
data['euclidean_distance'] = [euclidean(x,y) 
for (x,y) in zip(w2v_q1, w2v_q2)]
data['minkowski_distance'] = [minkowski(x,y,3) 
for (x,y) in zip(w2v_q1, w2v_q2)]
data['braycurtis_distance'] = [braycurtis(x,y) 
for (x,y) in zip(w2v_q1, w2v_q2)]

In [ ]:
fs4_1 = ['cosine_distance', 'cityblock_distance', 
         'jaccard_distance', 'canberra_distance', 
         'euclidean_distance', 'minkowski_distance',
         'braycurtis_distance']


In [ ]:
w2v = np.hstack((w2v_q1, w2v_q2))

In [ ]:
def wmd(s1, s2, model):
    s1 = str(s1).lower().split()
    s2 = str(s2).lower().split()
    stop_words = stopwords.words('english')
    s1 = [w for w in s1 if w not in stop_words]
    s2 = [w for w in s2 if w not in stop_words]
    return model.wmdistance(s1, s2)

In [ ]:
data['wmd'] = data.apply(lambda x: wmd(x['question1'], x['question2'], model), axis=1)
model.init_sims(replace=True) 
data['norm_wmd'] = data.apply(lambda x: wmd(x['question1'], x['question2'], model), axis=1)
fs4_2 = ['wmd', 'norm_wmd']

In [ ]:
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

In [ ]:
fs_1+fs_2+fs3_4+fs4_1+fs4_2

In [ ]:
import gc
import psutil
del([tfv_q1, tfv_q2, tfv, q1q2, 
     question1_vectors, question2_vectors, svd_q1, 
     svd_q2, q1_tfidf, q2_tfidf])
del([w2v_q1, w2v_q2])
del([model])
gc.collect()
psutil.virtual_memory()